In [1]:
import chess, glob, tqdm
import chess.pgn
from dataclasses import dataclass
import psycopg2
import chess.engine

conn = psycopg2.connect(
    database="xadrez",
    host="0.0.0.0",
    user="admin",
    password="admin",
    port="5051",
)

engine = chess.engine.SimpleEngine.popen_uci("stockfish/stockfish-ubuntu-x86-64")

In [2]:
# Função para avaliar uma posição usando Stockfish
def evaluate_position(fen, turn):
    info = engine.analyse(chess.Board(fen), chess.engine.Limit(time=0.1))
    
    score = info["score"].relative.score()
    
    if not turn:
        score = -score
        
    return score


def analyze_game(game):
    board = chess.Board()
    rows = []

    for move in game.mainline_moves():
        board.push(move)
        fen = board.fen()
        binary = board.board_fen()

        try:
            evaluation = evaluate_position(
                fen, board.turn
            )  # Você pode adicionar a lógica para avaliação aqui
            row = (fen, binary, float(evaluation))
        except:
            continue
        rows.append(row)

    return rows


def process_pgn(file_path):
    with open(file_path) as file:
        game = chess.pgn.read_game(file)
        rows = analyze_game(game)

        cursor = conn.cursor()
        for i, row in enumerate(rows, start=1):
            # print(f"{i}|{row}")
            try:
                pass
                cursor.execute(
                    "INSERT INTO estados_xadrez (fen, bin, eval) VALUES ('{}', '{}', {})".format(
                        row[0], row[1], row[2]
                    )
                )
                conn.commit()  # Certifique-se de commitar a transação se for bem-sucedida
            except Exception as e:
                conn.rollback()  # Faça o rollback da transação em caso de falha
                print(f"Erro durante a execução do SQL: {e}")

In [3]:
arquivos_pgn = glob.glob("download/*.pgn")
for i in tqdm.tqdm(arquivos_pgn):
    process_pgn(i)

100%|██████████| 247/247 [32:56<00:00,  8.00s/it]
